<a href="https://colab.research.google.com/github/AsifShaafi/Deep-Learning-Course-Examples/blob/main/RNN_Examples/Shaafi_CharacterGeneratorRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Natural Language Processing (Character Generation) with Recurrent Neural Network


We will generate charater using a RNN.  We will work with a dataset of Shakespeare's writing.  Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

- The model is character-based.
- The model is trained on small batches of text (100 characters each), and is able to generate a longer sequence of text with coherent structure.

*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*


In [1]:
# %tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Download the Shakespeare Dataset

Here, we use an extract from a Shadespheare play for training.  We can use our own text paragraph data and use it for network training.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# If you load your own data file, use the following lines:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

1115394/1115394 [==============================] - 0s 0us/step


###Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


###Encode
Encode each unique character as a different integer.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# Chect how a part of the text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Make a function that can convert the numeric values back to text.


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Create Training Examples

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform.

Our task is to feed the model a sequence and have it return the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [9]:
# Create a stream of characters from our text data
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Use the batch method to turn this stream of characters into batches of desired length
seq_length = 100  # length of sequence for a training example
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
# Use these sequences of length 101 and split them into input and output.
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(3):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


EXAMPLE

INPUT
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us k

OUTPUT
ow Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki


Make training batches.

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Build the Model
We use an embedding layer, a LSTM layer, and one dense layer that contains a node for each unique character in our training data.  The dense layer will give us a probability distribution over all nodes.

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    # tf.keras.layers.LSTM(rnn_units,
    #                     return_sequences=True,
    #                     stateful=True,
    #                     recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4021569 (15.34 MB)
Trainable params: 4021569 (15.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Create a Loss Function
Our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

We will create the loss function.

Before we do that, let's have a look at a sample input and the output from the untrained model.

In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.5398268e-03  1.9940408e-03 -7.8418781e-04 ...  1.5014500e-03
   -2.5000356e-03  6.8666367e-03]
  [-6.7877402e-03 -2.8799800e-03  8.8649448e-03 ...  1.1577982e-02
   -9.2960172e-04  2.3278143e-02]
  [-7.1512307e-03 -1.0209536e-02  9.4663957e-03 ... -5.1701297e-03
   -1.0460452e-02  2.6893567e-02]
  ...
  [ 4.4676811e-03  1.2351153e-02  8.0675259e-03 ... -1.2131406e-02
    9.4017470e-03  1.5520790e-02]
  [ 9.0279942e-03  9.6672531e-03  8.4042763e-03 ... -2.2407008e-02
   -8.5296258e-03  1.2490679e-02]
  [ 2.8900302e-03  4.1155033e-03  5.7223774e-03 ... -1.1332656e-02
   -6.1362353e-03  1.6967572e-02]]

 [[-2.5398268e-03  1.9940408e-03 -7.8418781e-04 ...  1.5014500e-03
   -2.5000356e-03  6.8666367e-03]
  [ 4.0361783e-03 -5.1465142e-03  1.0458693e-02 ... -4.5644557e-03
    1.5467219e-04  1.4952458e-02]
  [-2.0437567e-03 -1.0117114e-02 -8.0740796e-03 ...  8.2248375e-03
   -9.9001881e-03  7.2011454e-03]
  ...
  [-9.8121669e-03  1.7042361e-02  5.2523580e-03 ...  3.1379925e

In [15]:
# Let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00253983  0.00199404 -0.00078419 ...  0.00150145 -0.00250004
   0.00686664]
 [-0.00678774 -0.00287998  0.00886494 ...  0.01157798 -0.0009296
   0.02327814]
 [-0.00715123 -0.01020954  0.0094664  ... -0.00517013 -0.01046045
   0.02689357]
 ...
 [ 0.00446768  0.01235115  0.00806753 ... -0.01213141  0.00940175
   0.01552079]
 [ 0.00902799  0.00966725  0.00840428 ... -0.02240701 -0.00852963
   0.01249068]
 [ 0.00289003  0.0041155   0.00572238 ... -0.01133266 -0.00613624
   0.01696757]], shape=(100, 65), dtype=float32)


In [16]:
# and finally we look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.00253983  0.00199404 -0.00078419 -0.01611576 -0.00722464 -0.00157395
  0.00579915 -0.00153331 -0.00576487 -0.00115031 -0.0082573   0.00567606
  0.00399871 -0.00219784  0.00505316  0.0024156  -0.00032782 -0.00180424
 -0.00449704  0.00687823 -0.00955301  0.00205594 -0.01161763  0.00028385
 -0.0093211  -0.0113554  -0.00571568  0.00207564  0.01471475 -0.00698169
 -0.0121569  -0.00676271  0.00815114  0.01476621 -0.00822585 -0.00740254
  0.00838882  0.00716627 -0.00761428 -0.00630648 -0.00400001  0.01605946
 -0.00897437 -0.00055129  0.00486085 -0.00702544 -0.00722201  0.00829759
 -0.00038868  0.01861269  0.0027183   0.00266349 -0.00501958 -0.00306174
 -0.00893614  0.01002183  0.01053586  0.00936981 -0.0026834   0.01355232
 -0.00069789  0.00397704  0.00150145 -0.00250004  0.00686664], shape=(65,), dtype=float32)


In [17]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'iW:-PuX$HWrJ!Jon\n;HB,NmgvEXQT-pnTcEZr3OAadsiQE;KKTNQu.$&H--!JIKp&bJP!SSueGjY&t$wgp:D.gFIOwaYLbMCvfiz'

So, we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compile the Model
We can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.


In [19]:
model.compile(optimizer='adam', loss=loss)

###Create Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Train the Model
**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [21]:
history = model.fit(data, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 15s 58ms/step - loss: 2.6703
Epoch 2/30
172/172 [==============================] - 11s 56ms/step - loss: 1.9654
Epoch 3/30
172/172 [==============================] - 11s 57ms/step - loss: 1.6991
Epoch 4/30
172/172 [==============================] - 13s 59ms/step - loss: 1.5501
Epoch 5/30
172/172 [==============================] - 13s 61ms/step - loss: 1.4608
Epoch 6/30
172/172 [==============================] - 12s 60ms/step - loss: 1.3999
Epoch 7/30
172/172 [==============================] - 12s 58ms/step - loss: 1.3519
Epoch 8/30
172/172 [==============================] - 11s 57ms/step - loss: 1.3127
Epoch 9/30
172/172 [==============================] - 11s 56ms/step - loss: 1.2782
Epoch 10/30
172/172 [==============================] - 11s 57ms/step - loss: 1.2450
Epoch 11/30
172/172 [==============================] - 12s 59ms/step - loss: 1.2126
Epoch 12/30
172/172 [==============================] - 12s 61ms/step - loss: 1.1811
E

###Load the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [22]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model finishes training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [23]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [24]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generate Text
Generate some text using any starting string we choose.

Generating text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.  Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

![alt text](https://www.tensorflow.org/text/tutorials/images/text_generation_sampling.png)


In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Real Madrid will win Champions league
Real Madrid will win Champions league intent
And arges no other. Allarl.
Farewell, good battle; come buy, if thou stir more gold enough and
Did the importance to the ring.

Servant:
Why, what, ho? you, if thou art not nerephy,
To see this kindness struck whilst he were come.

Second Servant:
I may, sir.

ANGELO:
Shall say, sir, my horse! advise you, my lord,
I am call made but by liberal malice,
For we have laid it: bring o' the tomb;
And been so soft as you,
We are treason Edward's custom, sir:
Was it a woman's shame?
Would he holds behind this fault, I hear them, taken
The news.
Have you colder forth too true heir:'

Nurse:
Sir, it is a party time?

MIRANDA:
Besurph, I speak a word.

AIFIA:
Nay, ha, hear me build a hoardee till
my sorrows with one that way's fair, and all my banish people:
Cushing me me again, grave men, an


##Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. “Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core.” TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
3. “Understanding LSTM Networks.” Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.